# Projeto de IoT Básico

Conceitos:
- O que é IoT (Internet das Coisas)
- Como utilizar uma Unidade de MicroControlador (MCU)
- O que é um System on a Chip (SoC)
- Como programar em Python
- Conceito de Sockets
- Desenvolvimento HTML Básico

## Passo 1 - Conectar o computador ao NodeMCU

A conexão acontece pela [portal serial](1) USB \
[1]: [porta serial](https://pt.wikipedia.org/wiki/Interface_serial)

In [ ]:
%serialconnect to --port=/dev/cu.usbserial-1130 --baud=9600

: 

Testar se estamos conectados aos nosso NodeMCU

In [ ]:
print("Olá mundo")

Olá mundo


## Passo 2 - Desenvolvendo o nosso programa

Precisamos importar as bibliotecas (pedaços de código extra) que nos ajudam a fazer nosso programa.

Primeiro chamamos (importamos) as Bibliotecas básicas de configuração do MCU.

In [ ]:
# Desabilita as funções de DEBUG - encontrar bugs
import esp
esp.osdebug(None)

# Limpa a memória do nosso MCU para deixar espaço para o novo código
import gc
gc.collect()

Conectamos nosso MCU a Internet e pedimos que ele nos diga qual o endereço IP dele.

In [ ]:
import network # Biblioteca da Internet

# Dados de acesso ao WiFi
ssid = 'ALHN-9036'
password = 'QPebX2E@xS'

# Coloco o MCU em estado de Estação
station = network.WLAN(network.STA_IF)

# Ativamos o estado de estação
station.active(True)

# Estabelecemos a conexão com a Internet
station.connect(ssid, password)

# Este código se certifica que o MCU se conectou antes de continuar o código
while station.isconnected() == False:
    """ enquanto o método isconnected retornar Falso
    volte ao Loop sem fazer nada (pass) """
    pass

# Nos informa que a coenxão aconteceu
print('Connection successful')

# Nos mostra o endereço IP do Node
print(station.ifconfig())

Connection successful
('192.168.1.7', '255.255.255.0', '192.168.1.254', '192.168.1.254')


Agora vamos fazer nossa página Web e criar um servidor interno no MCU que vai receber a conexão do celular.

Além disto, precisamos dizer o que queremos fazer no MCU. No nosso caso, só acender um LED que existe nele.

In [ ]:
""" A biblioteca socket cria uma porta no MCU que fica "escutando" se alguém está querendo se conectar a ela.
    Ela cria a conexão para o nosso servidor.
 """
try:
  import usocket as socket
except:
  import socket

Este próximo código nos capacita a ler ou escrever nos pinos digitais do MCU.

O NodeMCU possui 16 pinos digitais e 1 analógico, além de portas de energia de 3.3v

In [ ]:
# Biblioteca machine me dá as possibilidades de conexão aos pinos do MCU
from machine import Pin

""" 
Aqui estou criando uma variável que vai representar o Pino 2. 
Preciso dizer como quero usar este pino, neste caso, quero usar ele para saída (OUT) de energia.
Como é uma porta digital, tenho somente 2 estados - ON (1) e OFF (0)
"""
led = Pin(2, Pin.OUT)

Antes de mais nada, vamos criar nossa página Web  que servirá como Dashboard de Controle.
Ela é um código HTML simples que inicia perguntando ao MCU se o LED tá ligado e nos mostra isso

In [ ]:
def web_page():
  if led.value() == 0:
    gpio_state="ON"
  else:
    gpio_state="OFF"
  
  html = """
        <html><head>
          <title>Web Server da UNIRUY</title>
          <meta name="viewport" content="width=device-width, initial-scale=1">
          <link rel="icon" href="data:,">
          <style>
          html{
            font-family: Helvetica;
            display:inline-block;
            margin: 0px auto;
            text-align: center;}
            header {
                display: inline-flexbox;
                margin-left: 10%;
                margin-right: 10%;
            }
        
          h1{
            color: #fc2603;
            padding: 2vh;}
          p{
            font-size: 1.5rem;}
          .button{
            display: inline-block; 
            background-color: #e7bd3b; 
            border: none; 
            border-radius: 4px; 
            color: white; 
            padding: 16px 40px;
            text-decoration: none;
            font-size: 30px;
            margin: 2px;
            cursor: pointer;}
          .button2{
            background-color:#4286f4;}
            img {
                float: right;
            }
          </style>
          </head>
          <body>
            <header>
                <img src="logo.jpg" width="10%">
                <h1>Dashboard de Controle</h1>
            </header>
            
            
          <p>Estado LED: <strong>""" + gpio_state + """</strong></p>
          <p><a href="/?led=on"><button class="button">ON</button></a></p>
          <p><a href="/?led=off"><button class="button button2">OFF</button></a></p>
            
            </body>
        </html>
"""
  return html

Aqui inicia nosso Servidor Web.

In [ ]:
# Iniciamos o socket com um tipo STREAM TCP (para conexões Web)
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Dizemos a ele para se conectar através da porta 80 - portão padrão da Web
s.bind(('', 80))

# E dizemos que ele pode aceitar até 5 conexões simultâneas
s.listen(5)


while True: # Loop eterno até fecharmos o Servidor (desligar da energia)
  # O método accept aceita a conexão do cliente e cria um novo socket conn e salva o endereço do cliente
  conn, addr = s.accept()
    # mostramos o endereço do cliente
  print('Got a connection from %s' % str(addr))

    # Recebemos (recv) do cliente o que ele quer nos mandar
  request = conn.recv(1024)
  request = str(request)
    # Aqui mostramos o que o cliente nos madou
  print('Content = %s' % request)
    # Vemos se o cliente apertou o botão ON ou OFF pelo endereço da página
  led_on = request.find('/?led=on')
  led_off = request.find('/?led=off')

    # de acordo com o botão apertado, ligamos ou desligamos o LED
  if led_on == 6:
    print('LED ON')
    led.value(0)
  if led_off == 6:
    print('LED OFF')
    led.value(1)
      # Aqui mandamos a resposta ao cliente com a página da Web
  response = web_page()
  conn.send('HTTP/1.1 200 OK\n')
  conn.send('Content-Type: text/html\n')
  conn.send('Connection: close\n\n')
  conn.sendall(response)
  conn.close()

.Got a connection from ('192.168.1.4', 41158)
Content = b'GET /?led=on HTTP/1.1\r\nHost: 192.168.1.7\r\nConnection: keep-alive\r\nDNT: 1\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Mobile Safari/537.36\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nReferer: http://192.168.1.7/?led=off\r\nAccept-Encoding: gzip, deflate\r\nAccept-Language: pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7,it-IT;q=0.6,it;q=0.5\r\n\r\n'
LED ON
Got a connection from ('192.168.1.4', 41160)
Content = b'GET /logo.jpg HTTP/1.1\r\nHost: 192.168.1.7\r\nConnection: keep-alive\r\nUser-Agent: Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Mobile Safari/537.36\r\nDNT: 1\r\nAccept: image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8\r\nReferer: http://192.168.1.7/?led=on\r\nAccept-E